<b>DISCLAUMER: </b> Due to working on two systems for ideation and execution, it was hard compiling them back. I was under the impression the codes submitted were only to verify cheating, therefore, I didn't incoporated towards my approach during the competition. I am sorry for the inconvinience, but I have summarized my code in this file and I have attached all my work in the zip file, please find it attached.

# 0. Import packages

In [ ]:
import os

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# 1. Load Base Dataset

In [ ]:
INPUT_DIR=''

In [ ]:
## Do not change this cell. 
## The reason for this is to prevent cheating using the original data from DACON. 
## If an assertion error occurs in that cell when TAs evaluate the submitted code of each student, it is considered cheating.

train = pd.read_csv(os.path.join(INPUT_DIR, 'train.csv'))
test = pd.read_csv(os.path.join(INPUT_DIR, 'test.csv'))
park = pd.read_csv(os.path.join(INPUT_DIR, 'park.csv'))
dcc = pd.read_csv(os.path.join(INPUT_DIR, 'day_care_center.csv'))

assert train.shape[0] == 329690 and train.shape[1] == 13, 'Do not change the format of the input data.'
assert test.shape[0] == 85097 and test.shape[1] == 12, 'Do not change the format of the input data.'
assert park.shape[0] == 1359 and park.shape[1] == 7, 'Do not change the format of the input data.'
assert dcc.shape[0] == 7373 and dcc.shape[1] == 10, 'Do not change the format of the input data.'

# 2. Load additional dataset
- Explain here the additional dataset that you used.
- If you do not use any additional dataset, it's ok and keep empty the below cell.

In [ ]:
dcc

# 3. EDA and Preprocessing
- Conduct exploratory data analysis and explain the insights from your analyses.
- Conduct data preprocessing, including feature engineering, and explain your insights that derive such techniques.

### PARK AND DCC

In [ ]:
dc = dcc.copy()
dc = dc.sort_values(by=['gu','city'])
# dc = dc.fillna(method='backfill')
# dc = dc.fillna(0,inplace=True)

In [ ]:
dc

In [ ]:
dc = dc.groupby(['gu','city']).sum().reset_index()

In [ ]:
park_dc = pd.merge(park,dc,on=['gu','city'],how='left')
park_dc = park_dc.sort_values(by=['gu','city'])
park_dc = park_dc.drop(['park_open_year'])

In [ ]:
dong_gu = park_dc.groupby(['gu','city'])['dong'].unique()
dong_gu = pd.DataFrame(dong_gu)
dong_gu = dong_gu.reset_index()
dong_gu = dong_gu.explode('dong')

In [ ]:
dong_gu = dong_gu.drop_duplicates(subset = ['city','dong'])

In [ ]:
park_dc.isna().sum()

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
park_dc[['day_care_baby_num','day_care_teacher_num','day_care_baby_per_teacher']] = imputer.fit_transform(park_dc[['day_care_baby_num','day_care_teacher_num','day_care_baby_per_teacher']])

In [ ]:
park_dc = park_dc.drop(['park_name'], axis=1)
park_dc = pd.get_dummies(park_dc, columns=['park_type'])
park_dc = park_dc.groupby(['gu','city']).sum().reset_index()

### PREPROCESSING TRAINING AND TESTING DATA

In [ ]:
train = pd.merge(train,dong_gu[['gu','city','dong']],on=['dong','city'],how='left')
test = pd.merge(test,dong_gu[['gu','city','dong']],on=['dong','city'],how='left')

park_dc = park_dc.drop(['dong'],axis=1)

train = pd.merge(train,park_dc,on=['gu','city'],how='left')
test = pd.merge(test,park_dc,on=['gu','city'],how='left')

In [ ]:
def preprocess(df_train):
    df_train['month_end'] = np.where(df_train['transaction_day'].isin(df_train['transaction_day'].unique()[-4:]),1,0)
    df_train['month_mid'] = np.where(df_train['transaction_day'] == df_train['transaction_day'].unique()[0],1,0)
    df_train['month_start'] = np.where(df_train['transaction_day'] == df_train['transaction_day'].unique()[1],1,0)
    df_train.drop(columns=['transaction_day'],axis=1,inplace=True)

    df_train['building_age'] = df_train['transaction_year'] - df_train['built_year']

    df_train['total_amenities_child'] = np.log(df_train['day_care_baby_num'] + df_train['teacher_num']+df_train['nursing_room_num']
                                         +df_train['playground_num']+df_train['CCTV_num'])
    
    # df_train['total_amenities_child_percentile'] = df_train['total_amenities_child'].rank(pct=True)



    df_train['teacher_per_child'] = df_train['teacher_num']/df_train['day_care_baby_num']
    # df_train['teacher_per_child_percentile'] = df_train['teacher_per_child'].rank(pct=True)

    #use KNN to fill missing values of lat and long
    from sklearn.impute import KNNImputer
    imputer = KNNImputer(n_neighbors=5)
    df_train[['lat','long']] = imputer.fit_transform(df_train[['lat','long']])
    df_train['lat'] = df_train['lat'].round(6)
    df_train['long'] = df_train['long'].round(6)

    # df_train['lat_norm'] = df_train.groupby('gu').lat.transform(lambda x: (x-x.mean())/x.std())
    # df_train['long_norm'] = df_train.groupby('gu').long.transform(lambda x: (x-x.mean())/x.std())
    # df_train = df_train.drop(columns=['lat','long'],axis=1)

    # df_train['height_percentile_gu'] = df_train.groupby('gu').floor.transform(lambda x: round(x/x.max(),3))
    df_train['height_percentile_dong'] = df_train.groupby('dong').floor.transform(lambda x: round(x/x.max(),3))
    df_train['floor_percentile'] = df_train.groupby('apartment_id').floor.transform(lambda x: round(x/x.max(),3))

    # df_train['house_area_percentile_gu'] = df_train.groupby('gu').house_area.transform(lambda x: round(x/x.mean(),3))
    # df_train['house_area_percentile_dong'] = df_train.groupby('dong').house_area.transform(lambda x: round(x/x.mean(),3))
    df_train['house_area_percentile'] = df_train.groupby('apartment_id').house_area.transform(lambda x: round(x/x.mean(),3))

    

    #LOG TRANSFORM
    #day_care_baby_num, teacher_num, nursing_room_num, playground_num, CCTV_num divided 
    
    # df_train['day_care_baby_num'] = np.log(df_train['day_care_baby_num']+1)
    # df_train['teacher_num'] = np.log(df_train['teacher_num']+1)
    # df_train['nursing_room_num'] = np.log(df_train['nursing_room_num']+1)
    # df_train['playground_num'] = np.log(df_train['playground_num']+1)
    # df_train['CCTV_num'] = np.log(df_train['CCTV_num']+1)
    # df_train['park_area'] = np.log(df_train['park_area']+1)
    # df_train['house_area'] = np.log(df_train['house_area'])

    df_train['city'] = np.where(df_train['city']=='seoul',1,0)

    return df_train

In [ ]:
df = pd.concat([train,test],axis=0)
df_train = train
df_test = test

In [ ]:
df = preprocess(df)

In [ ]:
train = df[df['index'].isin(df_train['index'])]
test = df[df['index'].isin(df_test['index'])]

In [ ]:
train.to_csv('data/train.csv')
test.to_csv('data/test.csv')

# 4. Modeling

In [ ]:
df_train = pd.read_csv('data/train.csv', index_col=0)
df_test = pd.read_csv('data/test.csv', index_col=0)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train['dong'] = df_train['dong'].astype(int)
df_test['dong'] = df_test['dong'].astype(int)

In [ ]:
y = df_train['PRICE']
X = df_train.drop(['index','PRICE'],axis=1)


In [ ]:
def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

X,y = shuffle_in_unison(np.array(X),np.array(y))

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping


normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(X))

regularizer = 0.005

model = keras.Sequential([
    normalizer,

    layers.Dense(512, activation='relu', kernel_regularizer=l2(regularizer)),
    layers.Dense(256, activation='relu', kernel_regularizer=l2(regularizer)),
    layers.Dense(256, activation='relu', kernel_regularizer=l2(regularizer)),
    layers.Dense(128, activation='relu', kernel_regularizer=l2(regularizer)),
    layers.Dense(128, activation='relu', kernel_regularizer=l2(regularizer)),
    layers.Dense(64, activation='relu', kernel_regularizer=l2(regularizer)),
    layers.Dense(64, activation='relu', kernel_regularizer=l2(regularizer)),
    layers.Dense(32, activation='relu', kernel_regularizer=l2(regularizer)),
    layers.Dense(32, activation='relu', kernel_regularizer=l2(regularizer)),


    
    layers.Dense(1)
])

model.compile(loss='mean_squared_error',metrics=['mean_absolute_error'],
                optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                )

reduce_lr = ReduceLROnPlateau(monitor='val_mean_absolute_error', factor=0.5,
                              # monitor='mean_absolute_error', factor=0.2,
                              patience=3, min_lr=0.0001)

early_stopping = EarlyStopping(
    monitor='val_mean_absolute_error',
    patience=5,
    restore_best_weights=True,
)


model.fit(X, y, 
                    validation_split=0.05,
                    epochs=50, 
                    callbacks=[reduce_lr, early_stopping],
                    batch_size=128
                    )

early_stopping = EarlyStopping(
    monitor='val_mean_absolute_error',
    patience=5,
    restore_best_weights=True,
)

reduce_lr = ReduceLROnPlateau(monitor='val_mean_absolute_error', factor=0.5,
                              # monitor='mean_absolute_error', factor=0.2,
                              patience=3, min_lr=0.00001)

model.fit(X, y, 
                    validation_split=0.05,
                    epochs=50, 
                    callbacks=[reduce_lr, early_stopping],
                    batch_size=128
                    )

model.fit(X, y, 
                    validation_split=0.05,
                    epochs=50, 
                    callbacks=[reduce_lr, early_stopping],
                    batch_size=128
                    )

# 5. Make submission

In [ ]:
test = df_test
X_test = test.drop(['index','PRICE'],axis=1)
X_test = X_test.astype('float32')
X_test = X_test.to_numpy()
test['PRICE'] = model.predict(X_test)

test = test[['index','PRICE']]
test.to_csv('last_1_last_last.csv',index=False)